In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import pandas as pd
import time

# 코스메

In [151]:
# 데이터 불러오기
cosme = pd.read_csv('통합_코스메_데이터.csv', encoding='EUC-KR')

# 데이터셋 복제
cosme_df = cosme.copy(deep=True)

In [152]:
# 데이터셋 기본 정보 조회
print(cosme_df.info())
print(cosme_df.head(3))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 8 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   순위       3000 non-null   int64  
 1   상품ID     3000 non-null   int64  
 2   브랜드_KR   3000 non-null   object 
 3   제품명_EN   3000 non-null   object 
 4   카테고리_EN  3000 non-null   object 
 5   평점       3000 non-null   float64
 6   리뷰수_EN   3000 non-null   object 
 7   가격       3000 non-null   int64  
dtypes: float64(1), int64(3), object(4)
memory usage: 187.6+ KB
None
   순위      상품ID   브랜드_KR                    제품명_EN          카테고리_EN   평점  \
0   1  10189359  브이티코스메틱  CICA Daily Soothing Mask  Sheet mask pack  5.2   
1   2  10145207       더샘         CP Chip Concealer        Concealer  4.6   
2   3  10176465       롬앤        Juicy Lasting Tint         lipstick  4.9   

          리뷰수_EN    가격  
0  10034 reviews  2420  
1   8804 reviews   858  
2   6709 reviews  1320  


## 기본 전처리

In [153]:
cosme_df[['평점','리뷰수_EN','가격']]

,평점,리뷰수_EN,가격
0,5.2,10034 reviews,2420
1,4.6,8804 reviews,858
2,4.9,6709 reviews,1320
3,4.8,"6,530 reviews",2970
4,5.2,"6,457 reviews",1090
...,...,...,...
2995,6.0,17 reviews,990
2996,5.1,17 reviews,1430
2997,5.0,17 reviews,1980
2998,4.9,17 reviews,1320


In [154]:
max_len = cosme_df['리뷰수_EN'].str.len().max()

cosme_df[cosme_df['리뷰수_EN'].str.len()==max_len]

,순위,상품ID,브랜드_KR,제품명_EN,카테고리_EN,평점,리뷰수_EN,가격
0,1,10189359,브이티코스메틱,CICA Daily Soothing Mask,Sheet mask pack,5.2,10034 reviews,2420
3,4,10217471,티르티르,MASK FIT RED CUSHION,Liquid foundation,4.8,"6,530 reviews",2970
4,5,10245780,넘버즈인,No. 5 Shiratama Glutathione C Furikake Mask,Sheet mask pack,5.2,"6,457 reviews",1090
6,7,10217712,이니스프리,Retinol Deer Repair Serum,Beauty fluid,4.7,"5,574 reviews",3960
7,8,10198590,아누아,Dokudami 77 Soothing Toner,Lotion,5.0,"5,308 reviews",2380
9,10,10216004,넘버즈인,No. 3 Smooth care sheet mask,Sheet mask pack,5.0,"4,383 reviews",1090
11,12,10137755,메디힐,N.M.F Aqua Ampoule Mask JEX,Sheet mask pack,5.4,"4,131 reviews",356
15,16,2926413,미샤,BB Cream UV,Sunscreen and UV care (for face),4.7,"3,420 reviews",2626
23,24,10225136,코스알엑스,RX the Vitamin C23 Serum,Beauty fluid,5.0,"2,238 reviews",2480
24,25,10235552,Wonjungyo,Won Jeongyo Metal Shower Pencil,Gel and cream eyeshadow,5.2,"2,178 reviews",1650


In [155]:
# 1. 컬럼 타입 변경 : 리뷰수_EN(int)
# 1) 리뷰수_EN 컬럼의 ' reviews', ',', '??' 삭제

for i in range(len(cosme_df)):
  review = cosme_df['리뷰수_EN'][i]

  review_result = review.replace('reviews', '')
  review_result = review_result.replace(',', '')
  review_result = review_result.replace(' ', '')
  review_result = review_result.replace('??', '')
  
  cosme_df.loc[i, '리뷰수_EN'] = review_result

print(cosme_df.loc[:10, '리뷰수_EN'])

0     10034
1      8804
2      6709
3      6530
4      6457
5      5865
6      5574
7      5308
8      4704
9      4383
10     4267
Name: 리뷰수_EN, dtype: object


In [156]:
# 1. 컬럼 타입 변경 : 리뷰수_EN(int)
# 2) 컬럼 타입 변경

cosme_df['리뷰수_EN'] = cosme_df['리뷰수_EN'].apply(pd.to_numeric)

print(cosme_df['리뷰수_EN'].dtypes)
print(cosme_df.loc[:5, '리뷰수_EN'])

int64
0    10034
1     8804
2     6709
3     6530
4     6457
5     5865
Name: 리뷰수_EN, dtype: int64


In [157]:
# 2. 컬럼 이름 변경

cosme_df.columns = ['순위', '제품ID', '브랜드', '제품명', '소분류', '평점', '후기 수', '가격']
cosme_df.columns

Index(['순위', '제품ID', '브랜드', '제품명', '소분류', '평점', '후기 수', '가격'], dtype='object')

In [158]:
# 3. 플랫폼 컬럼 추가 및 순서 변경
# 원본 데이터셋에 전처리 되어있음

cosme_df['플랫폼'] = '코스메' 

a = cosme_df.columns[-1:].to_list()

ex_a = cosme_df.drop(columns = cosme_df.columns[[-1]]).columns[:].to_list()

reorder = a + ex_a
cosme_df = cosme_df[reorder]

cosme_df.columns

Index(['플랫폼', '순위', '제품ID', '브랜드', '제품명', '소분류', '평점', '후기 수', '가격'], dtype='object')

## 대분류 및 중분류 통합

In [159]:
# 1. 중분류1 & 중분류3 컬럼 생성
cosme_df['중분류1'] = ''
print(cosme_df['중분류1'].head())

cosme_df['중분류3'] = ''
print(cosme_df['중분류3'].head())

0    
1    
2    
3    
4    
Name: 중분류1, dtype: object
0    
1    
2    
3    
4    
Name: 중분류3, dtype: object


In [160]:
# 기존 분류 unique 값 확인
print(cosme_df['소분류'].unique()) # -> 제품명에 포함된 이름으로 찾기

['Sheet mask pack' 'Concealer' 'lipstick' 'Liquid foundation'
 'Brow mascara' 'Beauty fluid' 'Lotion' 'Loose powder' 'Face cream'
 'Sunscreen and UV care (for face)' 'Powder eyeshadow' 'mascara'
 'Lip care and lip balm' 'Washing packs and masks'
 'Gel and cream eyeshadow' 'Presto powder' 'Other facial cleansers'
 'Oil cleansing' 'Shampoo and conditioner' 'Mist-like lotion'
 'Booster/Introducing Fluid' 'Other cleansing' 'Lip gloss'
 'Eye Care Eye Cream' 'Other foundations' 'Other skin care'
 'Pencil eyeliner' 'Makeup base' 'Liquid eyeliner' 'Gomage Peeling'
 'Liquid Cleansing' 'Manicure' 'Skincare kit' 'Hair packs and treatments'
 'Powder Cheek' 'Out-of-bath treatment' 'Eyebrow pencil'
 'Gel and cream blush' 'Face wash foam' 'Milk' 'Puff sponge'
 'Makeup brush' 'False nails and nail tips'
 'Perfumes and Fragrances (Other)' 'Face Oil Balm' 'Eyelash beauty liquid'
 'Powder foundation' 'Cream and gel foundation' 'Body soap'
 'Body lotion/milk' 'Powder eyebrow' 'Other body care' 'Gel Eyelin

- 삭제할 소분류 검색값: 'pouch', 'brush', 'body', 'Body', 'hair', 'Hair', 'Shampoo', 'perfume', 'Hand', 'hand', 'bath', 'Other skin care products', 'nail', 'Nail', 'Health', 'food', 'drink', 'supplement', 'Other eyeliner', 'Shaving remover', 'Other makeup products', 'Manicure', 'Appliance', 'Leg', 'Foot', 'Travel', 'Other kit', 'Puff', 'cotton', 'Medical', 'Scalp', 'others'
- 삭제할 브랜드 : 'Mijangsen'(헤어케어브랜드), 'Stilla'(한국 브랜드 아님), 'Soon Plus'(단백질파우더 팔고 있음)

In [161]:
# 불필요한 행 삭제하기
# 1) 소분류 값 검색

word_list = ['pouch', 'brush', 'body', 'Body', 'hair', 'Hair', 'Shampoo', 'shampoo', 'perfume', 'Perfumes', 'Hand', 'hand', 'bath', 'Other skin care products', 'nail', 'Nail', 'Health', 'food', 'Food', 'drink', 'Drink', 'supplement', 'Other eyeliner', 'Shaving remover', 'Other makeup products', 'Manicure', 'Appliance', 'Leg', 'Foot', 'Travel', 'Other kit', 'Puff', 'cotton', 'Medical', 'Scalp', 'others', 'Blotting']
word = '|'.join(word_list)

word_idx = cosme_df[cosme_df['소분류'].str.contains(word)].index

cosme_df = cosme_df.drop(word_idx)
cosme_df.info() # 전체행 3000개 -> 2671개로 감소

<class 'pandas.core.frame.DataFrame'>
Index: 2671 entries, 0 to 2999
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   플랫폼     2671 non-null   object 
 1   순위      2671 non-null   int64  
 2   제품ID    2671 non-null   int64  
 3   브랜드     2671 non-null   object 
 4   제품명     2671 non-null   object 
 5   소분류     2671 non-null   object 
 6   평점      2671 non-null   float64
 7   후기 수    2671 non-null   int64  
 8   가격      2671 non-null   int64  
 9   중분류1    2671 non-null   object 
 10  중분류3    2671 non-null   object 
dtypes: float64(1), int64(4), object(6)
memory usage: 250.4+ KB


In [ ]:
# 불필요한 행 삭제하기
# 2) 불필요한 브랜드 삭제

brand_list = ['Mijangsen', 'Stilla', 'Soon Plus'] 
brand = '|'.join(brand_list)

brand_idx = cosme_df[cosme_df['브랜드'].str.contains(brand)].index

cosme_df = cosme_df.drop(brand_idx)
cosme_df.info() # 전체행 2673개 -> 2581개로 감소

<class 'pandas.core.frame.DataFrame'>
Index: 2581 entries, 0 to 2999
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   플랫폼     2581 non-null   object 
 1   순위      2581 non-null   int64  
 2   제품ID    2581 non-null   int64  
 3   브랜드     2581 non-null   object 
 4   제품명     2581 non-null   object 
 5   소분류     2581 non-null   object 
 6   평점      2581 non-null   float64
 7   후기 수    2581 non-null   int64  
 8   가격      2581 non-null   int64  
 9   중분류1    2581 non-null   object 
 10  중분류3    2581 non-null   object 
dtypes: float64(1), int64(4), object(6)
memory usage: 242.0+ KB


In [163]:
# 불필요한 행 삭제하기
# 3) 인덱스 리셋

cosme_df.reset_index(drop=True, inplace=True)
cosme_df.head()

,플랫폼,순위,제품ID,브랜드,제품명,소분류,평점,후기 수,가격,중분류1,중분류3
0,코스메,1,10189359,브이티코스메틱,CICA Daily Soothing Mask,Sheet mask pack,5.2,10034,2420,,
1,코스메,2,10145207,더샘,CP Chip Concealer,Concealer,4.6,8804,858,,
2,코스메,3,10176465,롬앤,Juicy Lasting Tint,lipstick,4.9,6709,1320,,
3,코스메,4,10217471,티르티르,MASK FIT RED CUSHION,Liquid foundation,4.8,6530,2970,,
4,코스메,5,10245780,넘버즈인,No. 5 Shiratama Glutathione C Furikake Mask,Sheet mask pack,5.2,6457,1090,,


스킨케어 - 마스크팩, 클렌징, 토너/토너패드/스킨, 에센스/세럼/앰플, 로션, 크림, 기타
1) 마스크팩: (소)'Sheet mask pack'
2) 클렌징: (소)'wash', (소)'Peeling', (소)'clean', (소)'Clean', (소)'Remover'

3) 토너/토너패드/스킨
    - (소)'Lotion' => 이름에 'toner', 'Toner', 'pad', 'Pad','water', 'Water', 'skin', 'Skin', 'Shigeakisui'(설화수 제품명), 'mask', 'Mask' 들어가는 경우

4) 에센스/세럼/앰플: (소)'Booster', (소)'Beauty fluid'
    - (소)'Lotion' => 이름에 'essence', 'Essence', 'Essener', 'serum', 'Serum', 'booster' 들어가는 경우

5) 로션: (소)'Milk'
    - (소)'Lotion' => 이름에 'lotion', 'Lotion' 들어가는 경우 & 1)~4)에서 필터링하고 남은 나머지 상품들들

6) 크림
    - (소)'Face cream' => 'BB' 제외 나머지 제품

7) 기타: (소)'mist', (소)'Oil Balm', (소)'Skincare kit', (소)'Other skin care', (소)'Eye Cream', (소)'All', (소)'Skincare kit'

베이스메이크업 - 선케어, 파운데이션/쿠션/BB/CC, 파우더/팩트/픽서, 블러셔/쉐이딩/하이라이터, 프라이머/베이스, 컨실러
1) 선케어: (소)'Sunscreen'

2) 파운데이션/쿠션/BB/CC: (소)'foundation'
    - (소)'Face cream' => 이름에 'BB' 들어갈 경우

3) 파우더/팩트/픽서:
    - (소)'Cream and gel foundation'=> 이름에 'pact' 들어가는 경우 / 이외 모두 파운데이션으로 분류
    - (소) 'Presto powder' => 이름에 'powder', 'Powder', 'finsh', 'Finish', 'pact', 'Pact' 들어가는 경우
    - (소)'Loose'

4) 블러셔/쉐이딩/하이라이터: (소)'blush', (소)'Cheek'
    - (소) 'Presto powder' => 이름에 'shading' 'Shading', 'highlighter', 'Highlighter', 'blusher', 'Blusher', 'brusher', 'Brusher', 'contoure', 'Contoure' 들어가는 경우

5) 프라이머/베이스: (소)'Makeup base'

6) 컨실러: (소)'Concealer'

색조메이크업 - 립메이크업, 아이섀도우, 아이라이너, 아이브로우, 마스카라, 속눈썹 영양제
1) 립메이크업
    - 립글로스: (소)'Lip gloss'
    - 립밤: (소)'lip balm'
    - 립스틱: (소)'lipstick'
    - 립라이너: (소)'Lip liner'
    - 립틴트: 립글로스, 립스틱 중에서 이름에 'tint' 포함된 경우

2) 아이섀도우: (소)'eyeshadow', (소)'Palettes'

3) 아이라이너: (소)'eyeliner'

4) 아이브로우: (소)'eyebrow', (소)'Eyebrow', (소)'Brow mascara'

5) 마스카라: (소)'mascara', (소)'Mascara'

6) 속눈썹 영양제: (소)'Eyelash'

In [164]:
# 중분류3

clean_list = ['wash', 'Peeling', 'Wash', 'clean', 'Clean', 'Remover']
cleansing = '|'.join(clean_list)
cond_clean = cosme_df['소분류'].str.contains(cleansing)

toner_list = ['toner', 'Toner', 'pad', 'Pad','water', 'Water', 'skin', 'Skin', 'Shigeakisui', 'mask', 'Mask']
toner = '|'.join(toner_list)
cond_toner = cosme_df['제품명'].str.contains(toner)

serum_list = ['essence', 'Essence', 'Essener', 'serum', 'Serum', 'booster', 'Booster', 'Beauty fluid']
serum = '|'.join(serum_list)
cond_serum = cosme_df['제품명'].str.contains(serum)
cond_serum2 = cosme_df['소분류'].str.contains(serum)

extra_list = ['mist', 'Mist', 'Oil Balm', 'Skincare kit', 'Other skin care', 'Eye Cream', 'All', 'Skincare kit']
extra = '|'.join(extra_list)
cond_extra = cosme_df['소분류'].str.contains(extra)
#---------------------------------------------------------#
cond_bb = cosme_df['제품명'].str.contains('BB')

powder_list = ['powder', 'Powder', 'finsh', 'Finish', 'pact', 'Pact']
powder = '|'.join(powder_list)
cond_powder = cosme_df['제품명'].str.contains(powder)

shading_list = ['shading' 'Shading', 'highlighter', 'Highlighter', 'blusher', 'Blusher', 'brusher', 'Brusher', 'contoure', 'Contoure', 'face cube']
shading = '|'.join(shading_list)
cond_shading = cosme_df['제품명'].str.contains(shading)

blusher_list = ['blush', 'Cheek']
blusher = '|'.join(blusher_list)
cond_blusher = cosme_df['소분류'].str.contains(blusher)

cond_sun = cosme_df['소분류'].str.contains('Sunscreen')
cond_found = cosme_df['소분류'].str.contains('foundation')
cond_loose = cosme_df['소분류'].str.contains('Loose')
cond_base = cosme_df['소분류'].str.contains('Makeup base')
#---------------------------------------------------------#
cond_tint = cosme_df['제품명'].str.contains('tint')
cond_balm = cosme_df['소분류'].str.contains('lip balm')
cond_liner = cosme_df['소분류'].str.contains('Lip liner')
#---------------------------------------------------------#
shadow_list = ['eyeshadow', 'Palettes']
shadow = '|'.join(shadow_list)
cond_shadow = cosme_df['소분류'].str.contains(shadow)

brow_list = ['eyebrow', 'Eyebrow', 'Brow mascara']
brow= '|'.join(brow_list)
cond_brow = cosme_df['소분류'].str.contains(brow)

masc_list = ['mascara', 'Mascara']
masc= '|'.join(masc_list)
cond_masc = cosme_df['소분류'].str.contains(masc)

line_list = ['eyeliner', 'Eyeliner']
eyeline = '|'.join(line_list)
cond_eyeline = cosme_df['소분류'].str.contains(eyeline)

lash_list = ['eyelash', 'Eyelash']
eyelash = '|'.join(lash_list)
cond_eyelash = cosme_df['소분류'].str.contains(eyelash)
#---------------------------------------------------------#

for i in range(0, len(cosme_df)):
  if cosme_df['소분류'][i] == 'Lotion':
    if cond_toner[i] == True :
      cosme_df.loc[i, '중분류3'] = '토너/토너패드/스킨'
    elif cond_serum[i] == True:
      cosme_df.loc[i, '중분류3'] = '에센스/세럼/앰플'
    else:
      cosme_df.loc[i, '중분류3'] = '로션'
  
  elif cosme_df['소분류'][i] == 'Face cream':
    if cond_bb[i] == True:
      cosme_df.loc[i, '중분류3'] = '파운데이션/쿠션/BB/CC'
    else:
      cosme_df.loc[i, '중분류3'] = '크림'

  elif cosme_df['소분류'][i] == 'Cream and gel foundation':
    if cond_powder[i] == True:
      cosme_df.loc[i, '중분류3'] = '파우더/팩트/픽서'
    else:
      cosme_df.loc[i, '중분류3'] = '파운데이션/쿠션/BB/CC'
  
  elif cosme_df['소분류'][i] == 'Presto powder':
    if cond_shading[i] == True:
      cosme_df.loc[i, '중분류3'] = '블러셔/쉐이딩/하이라이터'
    else:
      cosme_df.loc[i, '중분류3'] = '파우더/팩트/픽서'      
  
  elif (cosme_df['소분류'][i]=='Lip gloss') | (cosme_df['소분류'][i]=='lipstick') :
    if cond_tint[i] == True:
      cosme_df.loc[i, '중분류3'] = '립틴트'
    elif cosme_df['소분류'][i] == 'Lip gloss':
      cosme_df.loc[i, '중분류3'] = '립글로스'      
    else:
      cosme_df.loc[i, '중분류3'] = '립스틱'
      
  else:

    if cond_clean[i] == True:
      cosme_df.loc[i, '중분류3'] = '클렌징'
    
    elif cosme_df['소분류'][i] == 'Sheet mask pack':
      cosme_df.loc[i, '중분류3'] = '마스크팩'

    elif cond_serum2[i] == True:
      cosme_df.loc[i, '중분류3'] = '에센스/세럼/앰플'
    
    elif cosme_df['소분류'][i] == 'Milk':
      cosme_df.loc[i, '중분류3'] = '로션'
  
    elif cond_extra[i] == True:
      cosme_df.loc[i, '중분류3'] = '기타'
#---------------------------------------------------------#
    elif cond_sun[i] == True:
      cosme_df.loc[i, '중분류3'] = '선케어'
#---------------------------------------------------------#    
    elif cond_found[i] == True:
      cosme_df.loc[i, '중분류3'] = '파운데이션/쿠션/BB/CC'

    elif cond_blusher[i] == True:
      cosme_df.loc[i, '중분류3'] = '블러셔/쉐이딩/하이라이터'
    
    elif cond_loose[i] == True:
      cosme_df.loc[i, '중분류3'] = '파우더/팩트/픽서'
    
    elif cond_base[i] == True:
      cosme_df.loc[i, '중분류3'] = '프라이머/베이스'
    
    elif cosme_df['소분류'][i] == 'Concealer':
      cosme_df.loc[i, '중분류3'] = '컨실러'
#---------------------------------------------------------#    
    elif cond_balm[i] == True:
      cosme_df.loc[i, '중분류3'] = '립밤'
      
    elif cond_liner[i] == True:
      cosme_df.loc[i, '중분류3'] = '립라이너'    
#---------------------------------------------------------#   
    elif cond_shadow[i] == True:
      cosme_df.loc[i, '중분류3'] = '아이섀도우'
      
    elif cond_eyeline[i] == True:
      cosme_df.loc[i, '중분류3'] = '아이라이너'  

    elif cond_brow[i] == True:
      cosme_df.loc[i, '중분류3'] = '아이브로우'  
    
    elif cond_masc[i] == True:
      cosme_df.loc[i, '중분류3'] = '마스카라'

    elif cond_eyelash[i] == True:
      cosme_df.loc[i, '중분류3'] = '속눈썹영양제'
    
    else:
      cosme_df.loc[i, '중분류3'] = '분류없음'

In [165]:
# 고유값 수 확인인
print(cosme_df['중분류3'].unique())
print(cosme_df['중분류3'].nunique())
cosme_df.loc[:10, ['소분류', '중분류3']]

['마스크팩' '컨실러' '립스틱' '파운데이션/쿠션/BB/CC' '아이브로우' '에센스/세럼/앰플' '토너/토너패드/스킨'
 '파우더/팩트/픽서' '크림' '선케어' '아이섀도우' '마스카라' '립밤' '클렌징' '기타' '립글로스' '아이라이너'
 '프라이머/베이스' '블러셔/쉐이딩/하이라이터' '립틴트' '로션' '속눈썹영양제' '립라이너']
23


,소분류,중분류3
0,Sheet mask pack,마스크팩
1,Concealer,컨실러
2,lipstick,립스틱
3,Liquid foundation,파운데이션/쿠션/BB/CC
4,Sheet mask pack,마스크팩
5,Brow mascara,아이브로우
6,Beauty fluid,에센스/세럼/앰플
7,Lotion,토너/토너패드/스킨
8,Sheet mask pack,마스크팩
9,Sheet mask pack,마스크팩


In [166]:
# 중분류2 생성
cosme_df['중분류2'] = cosme_df['중분류3'].copy()
cosme_df['중분류2'].head(10)

0              마스크팩
1               컨실러
2               립스틱
3    파운데이션/쿠션/BB/CC
4              마스크팩
5             아이브로우
6         에센스/세럼/앰플
7        토너/토너패드/스킨
8              마스크팩
9              마스크팩
Name: 중분류2, dtype: object

In [167]:
# 중분류2 수정
# 중분류3('립틴트', '립라이너', '립글로스', '립밤', '립스틱스틱') => 중분류2(립메이크업) 으로 일괄 변경 

lip_list = ['립틴트', '립라이너', '립글로스', '립밤', '립스틱']
lips = '|'.join(lip_list)
cond_lips = cosme_df['중분류3'].str.contains(lips)

for i in range(len(cosme_df)):
  if cond_lips[i] == True:
    cosme_df.loc[i, '중분류2'] = '립메이크업'

In [168]:
print(cosme_df['중분류2'].unique())
print(cosme_df['중분류2'].nunique())
cosme_df.loc[:, ['소분류', '중분류2', '중분류3']]

['마스크팩' '컨실러' '립메이크업' '파운데이션/쿠션/BB/CC' '아이브로우' '에센스/세럼/앰플' '토너/토너패드/스킨'
 '파우더/팩트/픽서' '크림' '선케어' '아이섀도우' '마스카라' '클렌징' '기타' '아이라이너' '프라이머/베이스'
 '블러셔/쉐이딩/하이라이터' '로션' '속눈썹영양제']
19


,소분류,중분류2,중분류3
0,Sheet mask pack,마스크팩,마스크팩
1,Concealer,컨실러,컨실러
2,lipstick,립메이크업,립스틱
3,Liquid foundation,파운데이션/쿠션/BB/CC,파운데이션/쿠션/BB/CC
4,Sheet mask pack,마스크팩,마스크팩
...,...,...,...
2576,Lip care and lip balm,립메이크업,립밤
2577,Gel and cream eyeshadow,아이섀도우,아이섀도우
2578,Beauty fluid,에센스/세럼/앰플,에센스/세럼/앰플
2579,Pencil eyeliner,아이라이너,아이라이너


In [169]:
# 중분류1 - 중분류2
# 스킨케어 - 토너/토너패드/스킨, 에센스/세럼/앰플, 로션, 크림
# 베이스메이크업 - 파운데이션/쿠션/BB/CC, 파우더/팩트/픽서, 블러셔/쉐이딩/하이라이터, 프라이머/베이스, 컨실러
# 아이메이크업 - 아이섀도우, 아이라이너, 아이브로우, 마스카라, 속눈썹 영양제
# 클렌징 / # 마스크팩 / # 기타 / # 선케어 / # 립메이크업

skincare_list = ['토너/토너패드/스킨', '에센스/세럼/앰플', '로션', '크림']
skincare = '|'.join(skincare_list)
cond_skincare = cosme_df['중분류2'].str.contains(skincare)

base_list = ['파운데이션/쿠션/BB/CC', '파우더/팩트/픽서', '블러셔/쉐이딩/하이라이터', '프라이머/베이스', '컨실러']
base = '|'.join(base_list)
cond_base = cosme_df['중분류2'].str.contains(base)

eye_list = ['아이섀도우', '아이라이너', '아이브로우', '마스카라', '속눈썹영양제']
eye = '|'.join(eye_list)
cond_eye = cosme_df['중분류2'].str.contains(eye)

for i in range(len(cosme_df)):
  if cond_skincare[i] == True:
    cosme_df.loc[i, '중분류1'] = '스킨케어'
  
  elif cond_base[i] == True:
    cosme_df.loc[i, '중분류1'] = '베이스메이크업'

  elif cond_eye[i] == True:
    cosme_df.loc[i, '중분류1'] = '아이메이크업'
      
  else:
    if cosme_df['중분류2'][i] == '클렌징':
      cosme_df.loc[i, '중분류1'] = '클렌징'

    elif cosme_df['중분류2'][i] == '마스크팩':
      cosme_df.loc[i, '중분류1'] = '마스크팩'
      
    elif cosme_df['중분류2'][i] == '기타':
      cosme_df.loc[i, '중분류1'] = '기타'
      
    elif cosme_df['중분류2'][i] == '선케어':
      cosme_df.loc[i, '중분류1'] = '선케어'
    
    elif cosme_df['중분류2'][i] == '립메이크업':
      cosme_df.loc[i, '중분류1'] = '립메이크업'

In [170]:
print(cosme_df['중분류1'].unique())
print(cosme_df['중분류1'].nunique())
cosme_df.loc[:, ['소분류', '중분류1', '중분류2', '중분류3']]

['마스크팩' '베이스메이크업' '립메이크업' '아이메이크업' '스킨케어' '선케어' '클렌징' '기타']
8


,소분류,중분류1,중분류2,중분류3
0,Sheet mask pack,마스크팩,마스크팩,마스크팩
1,Concealer,베이스메이크업,컨실러,컨실러
2,lipstick,립메이크업,립메이크업,립스틱
3,Liquid foundation,베이스메이크업,파운데이션/쿠션/BB/CC,파운데이션/쿠션/BB/CC
4,Sheet mask pack,마스크팩,마스크팩,마스크팩
...,...,...,...,...
2576,Lip care and lip balm,립메이크업,립메이크업,립밤
2577,Gel and cream eyeshadow,아이메이크업,아이섀도우,아이섀도우
2578,Beauty fluid,스킨케어,에센스/세럼/앰플,에센스/세럼/앰플
2579,Pencil eyeliner,아이메이크업,아이라이너,아이라이너


In [171]:
# 대분류 - 중분류1
# 스킨케어 - 마스크팩, 클렌징, 기타, 스킨케어
# 베이스메이크업 - 선케어, 베이스메이크업
# 색조메이크업 - 립메이크업, 아이메이크업

skinclass_list = ['마스크팩', '클렌징', '기타', '스킨케어']
skinclass = '|'.join(skinclass_list)
cond_skinclass = cosme_df['중분류1'].str.contains(skinclass)

baseclass_list = ['선케어', '베이스메이크업']
baseclass = '|'.join(baseclass_list)
cond_baseclass = cosme_df['중분류1'].str.contains(baseclass)

colorclass_list = ['아이메이크업', '립메이크업']
colorclass = '|'.join(colorclass_list)
cond_colorclass = cosme_df['중분류1'].str.contains(colorclass)

for i in range(len(cosme_df)):
  if cond_skinclass[i] == True:
    cosme_df.loc[i, '대분류'] = '스킨케어'
  
  elif cond_baseclass[i] == True:
    cosme_df.loc[i, '대분류'] = '베이스메이크업'

  elif cond_colorclass[i] == True:
    cosme_df.loc[i, '대분류'] = '색조메이크업'

In [172]:
# 대분류 고유값 확인 
print(cosme_df['대분류'].unique())
print(cosme_df['대분류'].nunique())
cosme_df.loc[:, ['대분류', '소분류', '중분류1', '중분류2', '중분류3']]

['스킨케어' '베이스메이크업' '색조메이크업']
3


,대분류,소분류,중분류1,중분류2,중분류3
0,스킨케어,Sheet mask pack,마스크팩,마스크팩,마스크팩
1,베이스메이크업,Concealer,베이스메이크업,컨실러,컨실러
2,색조메이크업,lipstick,립메이크업,립메이크업,립스틱
3,베이스메이크업,Liquid foundation,베이스메이크업,파운데이션/쿠션/BB/CC,파운데이션/쿠션/BB/CC
4,스킨케어,Sheet mask pack,마스크팩,마스크팩,마스크팩
...,...,...,...,...,...
2576,색조메이크업,Lip care and lip balm,립메이크업,립메이크업,립밤
2577,색조메이크업,Gel and cream eyeshadow,아이메이크업,아이섀도우,아이섀도우
2578,스킨케어,Beauty fluid,스킨케어,에센스/세럼/앰플,에센스/세럼/앰플
2579,색조메이크업,Pencil eyeliner,아이메이크업,아이라이너,아이라이너


## 데이터셋 정리

In [173]:
# 기존 카테고리 지우기
cosme_df.drop(columns={'제품ID', '소분류'}, axis=1, inplace=True)
cosme_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2581 entries, 0 to 2580
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   플랫폼     2581 non-null   object 
 1   순위      2581 non-null   int64  
 2   브랜드     2581 non-null   object 
 3   제품명     2581 non-null   object 
 4   평점      2581 non-null   float64
 5   후기 수    2581 non-null   int64  
 6   가격      2581 non-null   int64  
 7   중분류1    2581 non-null   object 
 8   중분류3    2581 non-null   object 
 9   중분류2    2581 non-null   object 
 10  대분류     2581 non-null   object 
dtypes: float64(1), int64(3), object(7)
memory usage: 221.9+ KB


In [174]:
# 컬럼 순서 변경 : '플랫폼', '브랜드', '제품명', '중분류_순위', '대분류', '중분류1', '중분류2', '중분류3', '평점', '후기 수', '정가', '할인가'
a = ['순위', '대분류', '중분류1', '중분류2', '중분류3']

before_a = cosme_df.drop(columns = {'순위', '대분류', '중분류1', '중분류2', '중분류3'}).columns[:3].to_list()
after_a = cosme_df.drop(columns = {'순위', '대분류', '중분류1', '중분류2', '중분류3'}).columns[3:].to_list()

reorder = before_a + a + after_a
cosme_df = cosme_df[reorder]

cosme_df.columns

Index(['플랫폼', '브랜드', '제품명', '순위', '대분류', '중분류1', '중분류2', '중분류3', '평점', '후기 수',
       '가격'],
      dtype='object')

In [175]:
# 순위 재정립 : 인덱스 + 1

for i in range(0, len(cosme_df)):
  cosme_df['순위'][i] = i+1

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_30876\3200858311.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cosme_df['순위'][i] = i+1
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_30876\3200858311.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cosme_df['순위'][i] = i+1
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_30876\3200858311.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cosme_df['순위'][i

In [176]:
# 최종 데이터셋 확인
print(cosme_df.info())
cosme_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2581 entries, 0 to 2580
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   플랫폼     2581 non-null   object 
 1   브랜드     2581 non-null   object 
 2   제품명     2581 non-null   object 
 3   순위      2581 non-null   int64  
 4   대분류     2581 non-null   object 
 5   중분류1    2581 non-null   object 
 6   중분류2    2581 non-null   object 
 7   중분류3    2581 non-null   object 
 8   평점      2581 non-null   float64
 9   후기 수    2581 non-null   int64  
 10  가격      2581 non-null   int64  
dtypes: float64(1), int64(3), object(7)
memory usage: 221.9+ KB
None


,플랫폼,브랜드,제품명,순위,대분류,중분류1,중분류2,중분류3,평점,후기 수,가격
0,코스메,브이티코스메틱,CICA Daily Soothing Mask,1,스킨케어,마스크팩,마스크팩,마스크팩,5.2,10034,2420
1,코스메,더샘,CP Chip Concealer,2,베이스메이크업,베이스메이크업,컨실러,컨실러,4.6,8804,858
2,코스메,롬앤,Juicy Lasting Tint,3,색조메이크업,립메이크업,립메이크업,립스틱,4.9,6709,1320
3,코스메,티르티르,MASK FIT RED CUSHION,4,베이스메이크업,베이스메이크업,파운데이션/쿠션/BB/CC,파운데이션/쿠션/BB/CC,4.8,6530,2970
4,코스메,넘버즈인,No. 5 Shiratama Glutathione C Furikake Mask,5,스킨케어,마스크팩,마스크팩,마스크팩,5.2,6457,1090


In [ ]:
# 최종 데이터셋 저장
# cosme_df.to_csv('cosme_preprocessing.csv', index=False, encoding="utf-8-sig")